# Forest Cover Case Study


## Introduction:

Rasterframes is a powerful tool for combining geospatial data and spark dataframes. In this case study, we will walk through using rasterframes to conduct analysis of a forested region to determine the extent of deforestation using NDVI (Normalized Difference Vegetation Index) combined with basic machine learning techniques available through PySparkML. This tutorial assumes a basic knowledge of dataframes, more info can be found [here](https://spark.apache.org/docs/latest/sql-programming-guide.html#datasets-and-dataframes): 

## Background:

As humans expand across the planet, land that was once forested is cleared for other activities such as farming or logging. Such land is often high in biodiversity, and while our forests are an essential feature in our ecosystem, they are being destroyed at an unsustainable rate. Gathering data on the precise regions in which this destruction is occurring allows scientists and authorities to more easily combat deforestation. Satellite data can be utilized to track and prevent deforestation, as demonstrated by this tutorial, which uses an example from the Amazon rainforest.

## Initializing the environment:

First, some imports must be made. The rasterframes library must be imported as well as Apache Spark, which serves as the processing backbone of RasterFrames. 

In [2]:
import org.apache.spark.sql._
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import geotrellis.raster._

import org.apache.spark.sql._
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import geotrellis.raster._


Once the imports have been made, a `sparkSession` must be created. Note the `withRasterFrames()` method that is called which enables access to the rasterframes API.

In [3]:
implicit val spark = SparkSession.builder().
  master("local[*]").appName(getClass.getName).getOrCreate().withRasterFrames
spark.sparkContext.setLogLevel("ERROR")

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@14016ec0


## Reading in a Rasterframe

Once everything has been initialized, the next step is to read in our data. This example will use data in the form of a GeoTiff, but there are other ways to create a rasterframe, including from a geotrellis `Layer`, `ProjectedExtent`, or `tileLayerRDD` (see [creating-rasterframes](../Minis/creating-rasterframes-scala.html) for more info). In the case of a geotiff, it's as simple as pointing spark.read.geotiff to the input file(s).

In [4]:
import spark.implicits._

def readTiff(name: String): SinglebandGeoTiff = SinglebandGeoTiff(s"$name")

val filePath = "data/brazil_1/band2.tif"
val initRF = readTiff(filePath).projectedRaster.toRF(s"band_2")

import spark.implicits._
readTiff: (name: String)geotrellis.raster.io.geotiff.SinglebandGeoTiff
filePath: String = data/brazil_1/band2.tif
initRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_2: rf_tile]


## Initial analysis

Immediately, we can perform some basic functions on this rasterframe. A brief `rf.show()` will demonstrate that a rasterframe is very similar to a dataframe, with named columns corresponding to certain attributes. For instance, `tile` contains all the cell values of the image. `bounds` contains the bounds of the tiff's geometry, etc. In this way, a tiff raster can be represented as a rasterframe.

In [5]:
initRF.show()

+-----------+--------------------+
|spatial_key|              band_2|
+-----------+--------------------+
|      [0,0]|ShortUserDefinedN...|
+-----------+--------------------+



We can conduct some initial analysis on the tile by calling `tileStats` on the tile. The output is an array with the number of cells, number of noDataCells, min, max, mean, and variance respectively (for more on tile statistics, see [tile statistics](../Minis/statistics-scala.html)). 

In [5]:
initRF.select(tileStats($"band_2")).show(false)

+---------+-----------+-----+-----+-----------------+-----------------+
|dataCells|noDataCells|min  |max  |mean             |variance         |
+---------+-----------+-----+-----+-----------------+-----------------+
|51525    |-1         |125.0|726.0|237.8598544395925|4944.952683336407|
+---------+-----------+-----+-----+-----------------+-----------------+



We can examine the cell type of a tile with the `cellType` command.

In [6]:
initRF.select(cellType($"band_2")).show()

+--------------------------+
|celltypeexpression(band_2)|
+--------------------------+
|              int16ud-9999|
+--------------------------+



Individual stats about a tile can be obtained through the family of methods including `tileMean`, `tileSum`, `tileMin`, etc. You'll notice the output of `tileMean` is consistent with the mean value found with `tileStats`.

In [7]:
initRF.select(tileMean($"band_2")).show()

+------------------+
|  tileMean(band_2)|
+------------------+
|237.85985443959243|
+------------------+



## Loading the rest of the bands

This image is a multiband image, but we are going to load in each band individually and then perform a spatial join on the bands. This allows us to get all the bands in one rasterframe. For this demo, we are using visual and near infrared light to calculate NDVI and perform unsupervised clustering on the dataset. This means that we are loading in bands 2 through 5, which correspond to Blue, Green, Red, and NIR on the [Landsat 8 OLI](https://lta.cr.usgs.gov/L8). `spatialJoin` is a dataframe join that joins according to a spatial_key column. Bounds and metadata are discarded because they are no longer useful for the project.

This project analyzes three seperate scenes, all captured in close proximity to each other in the Brazilian rainforest. The background of this image is the green band, and it seems obvious from a brief look at this band that there is deforestation occurring at all three sites.

![Pic](pics/site_locs.png)

In [6]:
// Three scenes
val sceneNums = 1 to 3
// Four bands per scene (2, 3, 4, and 5)
val bandNums = 2 to 5

val filePattern = "data/brazil_%d/band%d.tif"                         

sceneNums: scala.collection.immutable.Range.Inclusive = Range(1, 2, 3)
bandNums: scala.collection.immutable.Range.Inclusive = Range(2, 3, 4, 5)
filePattern: String = data/brazil_%d/band%d.tif


Create the name of the files that we will need to read. Then, read all those files in and convert them to RasterFrames. Finally, perform a spatial join on the list of rasterframes containing only one band as a tile to create a larger RasterFrame.

In [7]:
val fullRFs = sceneNums.map{sn => bandNums.map { bn => (bn, filePattern.format(sn, bn))}
                   .map{bandFile => readTiff(bandFile._2).projectedRaster.toRF("band_%d".format(bandFile._1))}
                   .reduce(_ spatialJoin _)}

fullRFs: scala.collection.immutable.IndexedSeq[astraea.spark.rasterframes.RasterFrame] = Vector([spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 3 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 3 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 3 more fields])


As you can see, all four bands of all three scenes are now loaded into three seperate rasterframes.

In [10]:
for (rf <- fullRFs) rf.show()

+-----------+--------------------+--------------------+--------------------+--------------------+
|spatial_key|              band_2|              band_3|              band_4|              band_5|
+-----------+--------------------+--------------------+--------------------+--------------------+
|      [0,0]|ShortUserDefinedN...|ShortUserDefinedN...|ShortUserDefinedN...|ShortUserDefinedN...|
+-----------+--------------------+--------------------+--------------------+--------------------+

+-----------+--------------------+--------------------+--------------------+--------------------+
|spatial_key|              band_2|              band_3|              band_4|              band_5|
+-----------+--------------------+--------------------+--------------------+--------------------+
|      [0,0]|ShortUserDefinedN...|ShortUserDefinedN...|ShortUserDefinedN...|ShortUserDefinedN...|
+-----------+--------------------+--------------------+--------------------+--------------------+

+-----------+-----

In addition, it is possible to display the center of the extent of the tile in (Lat, Long) format. The method `withCenter` will display it in the native CRS, and `withBounds` and `withSpatialIndex` also exist.

In [11]:
fullRFs.apply(0).withCenterLatLng().select("center").show(1, false)

+---------------------------------------+
|center                                 |
+---------------------------------------+
|[-66.72875437318221,-8.829838122908235]|
+---------------------------------------+



## Feature Engineering

This example uses [NDVI](../Minis/ndvi-scala.html), which is a metric computed from the normalized difference between two of our bands, the NIR band (band 5), and the red band (band 4). We will add another column, this time with the tiles as the normalized difference between the NIR and red bands for each scene. In addition, it is sometimes useful to compute additional relationships between bands to use as features in a machine learning algorithm. Additionally, the cell types of the NIR and red bands are converted to floats for the purposes of stability with `convertCellType`, which changes the data type of the cells in a tile. The diagram below is a local function that operates on colors (See [Tile Arithmetic](../Minis/tile-arithmetic-scala.html) for more information on map algebra).

![-](../pics/local-functions.gif)

In [13]:
val ndviRFs = fullRFs.map { rf => rf.withColumn("ndvi",
        normalizedDifference(convertCellType($"band_5", "float32"), convertCellType($"band_4", "float32"))) }

ndviRFs: scala.collection.immutable.IndexedSeq[org.apache.spark.sql.DataFrame] = Vector([spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 4 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 4 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 4 more fields])


Normalized difference is a local function that is equivalent to $\frac{NIR - Red}{NIR + Red}$. Local functions perform functions on a cellwise basis. Every cell in a band has corresponding cells in another band, and the output of a local function is a tile where the cell values are determined by some function that operates on every cell individually. We will find the sum of the green and blue bands to use in our clustering model. `localAdd` is part of a family of arithemtic functions, and it adds the values of every corresponding cell together to produce an output cell. There also exist local scalar arithmetic functions, which you must append "Int" to to use with an integer.

In [16]:
val completeRFs = ndviRFs.map { rf => rf.withColumn("grvi",
        normalizedDifference(convertCellType($"band_3", "float32"), convertCellType($"band_4", "float32"))) }

completeRFs: scala.collection.immutable.IndexedSeq[org.apache.spark.sql.DataFrame] = Vector([spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 5 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 5 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 5 more fields])


In [17]:
for (rf <- completeRFs) rf.select(tileStats($"ndvi")).show(false)

+---------+-----------+------------------+------------------+------------------+--------------------+
|dataCells|noDataCells|min               |max               |mean              |variance            |
+---------+-----------+------------------+------------------+------------------+--------------------+
|51525    |-1         |0.2584196925163269|0.9273137450218201|0.8077649412792555|0.008396342462181239|
+---------+-----------+------------------+------------------+------------------+--------------------+

+---------+-----------+-------------------+-----------------+------------------+-------------------+
|dataCells|noDataCells|min                |max              |mean              |variance           |
+---------+-----------+-------------------+-----------------+------------------+-------------------+
|57072    |-1         |-0.5266343951225281|0.914093017578125|0.7854701653986633|0.01124415954480573|
+---------+-----------+-------------------+-----------------+------------------+-----

## Machine learning

The purpose of this demonstration is to determine the extent of deforestation in the selected areas in a procedural way. To this end, we will use a straightforward [k-means](https://en.wikipedia.org/wiki/K-means_clustering) clustering algorithm available to us through SparkML. K-means operates by establishing k cluster centers and then assigning every observation to a cluster based on the distance it is from the cluster center. The location of the cluster centers are moved to be the centroid of the observations, and the process is repeated until convergence occurs (For another example of clustering, see [clustering](../Minis/clustering.html)). 

These are the tile stats for the NDVI, GRVI, and infrared bands for the first scene.

In [22]:
completeRFs.apply(0).select(tileStats($"ndvi")).show()
completeRFs.apply(0).select(tileStats($"grvi")).show()
completeRFs.apply(0).select(tileStats($"band_5")).show()

+---------+-----------+------------------+------------------+------------------+--------------------+
|dataCells|noDataCells|               min|               max|              mean|            variance|
+---------+-----------+------------------+------------------+------------------+--------------------+
|    51525|         -1|0.2584196925163269|0.9273137450218201|0.8077649412792555|0.008396342462181239|
+---------+-----------+------------------+------------------+------------------+--------------------+

+---------+-----------+--------------------+------------------+-------------------+--------------------+
|dataCells|noDataCells|                 min|               max|               mean|            variance|
+---------+-----------+--------------------+------------------+-------------------+--------------------+
|    51525|         -1|-0.16288061439990997|0.3622449040412903|0.17350494746009204|0.009846214924387444|
+---------+-----------+--------------------+------------------+------

Recall that k-means uses the euclidean distance between two features to assign cluster values to each cell, which is the distance in 6 dimensional space where the six feature dimensions are the four bands and the two indexes we created. However, as we can see from the tile stats, ndvi and grvi are between 0 and 1 while the bands have values in the thousands. This means that the distance between any two points will be largely determined by the band values, which defeats the point of using our vegetation indexes. One solution is to scala the value of the indexes to be more significant.

In [28]:
val scaledRFs = completeRFs.map { rf => rf.withColumn("ndvi_s", localMultiplyScalar($"ndvi", 500.0))
    .withColumn("grvi_s", localMultiplyScalar($"grvi", 500.0)) }

scaledRFs: scala.collection.immutable.IndexedSeq[org.apache.spark.sql.DataFrame] = Vector([spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 7 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 7 more fields], [spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 7 more fields])


### Pipeline

The algorithm works through assigning features to clusters. PySparkML requires that each feature be in its own row, and those features be packed into a single `Vector`. This means that all the information must be packed into a single feature vector. The first step is to "explode" the tiles into a single row per cell/pixel, so the top left cell is assigned its own row, with the cellwise values of every band as the values in that row.

In [29]:
import astraea.spark.rasterframes.ml.TileExploder

val exploder = new TileExploder()

import astraea.spark.rasterframes.ml.TileExploder
exploder: astraea.spark.rasterframes.ml.TileExploder = tile-exploder_f88d8cae963c


In [30]:
val bandColNames = bandNums.map { x => "band_%d".format(x) }

bandColNames: scala.collection.immutable.IndexedSeq[String] = Vector(band_2, band_3, band_4, band_5)


The next step is for the cell values in their seperate columns to be assembled into a single feature vector. PySparkML has a tool for this called a `VectorAssembler` which has as inputs a certain number of columns of features and outputs a single column with the vectorized features in it.

In [34]:
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler()
    .setInputCols((bandColNames :+ ("ndvi") :+ ("grvi")).toArray)
    .setOutputCol("features")

import org.apache.spark.ml.feature.VectorAssembler
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_1a014a1a5fd9


For our example we are setting the number of clusters to 3, although the value is arbitrary.

In [35]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}

val kmeans = new KMeans().setK(3)

// We establish our pipeline with our stages
val pipeline = new Pipeline().setStages(Array(exploder, assembler, kmeans))

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_236f4caae1dd
pipeline: org.apache.spark.ml.Pipeline = pipeline_2734f75a963f


Finally, we fit our model and compute our clusters

In [36]:
val models = completeRFs.map { rf => pipeline.fit(rf) }

models: scala.collection.immutable.IndexedSeq[org.apache.spark.ml.PipelineModel] = Vector(pipeline_2734f75a963f, pipeline_2734f75a963f, pipeline_2734f75a963f)


Upon fitting the model and transforming the data, we see the exploded data. Instead of a tile, every entry in a band contain a single pixel value. In addition, there is a prediction column that contains the result of the model's clustering.

In [45]:
val rfModelZip = completeRFs.zip(models)

rfModelZip: scala.collection.immutable.IndexedSeq[(org.apache.spark.sql.DataFrame, org.apache.spark.ml.PipelineModel)] = Vector(([spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 5 more fields],pipeline_2734f75a963f), ([spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 5 more fields],pipeline_2734f75a963f), ([spatial_key: struct<col: int, row: int>, band_2: rf_tile ... 5 more fields],pipeline_2734f75a963f))


In [46]:
val clusteredDFs = rfModelZip.map { rf_md => rf_md._2.transform(rf_md._1) }

clusteredDFs.apply(0).drop("row_index", "features").show(3)

+-----------+------------+------+------+------+------+------------------+-------------------+----------+
|spatial_key|column_index|band_2|band_3|band_4|band_5|              ndvi|               grvi|prediction|
+-----------+------------+------+------+------+------+------------------+-------------------+----------+
|      [0,0]|           0| 188.0| 349.0| 206.0|2750.0|0.8606224656105042|0.25765764713287354|         0|
|      [0,0]|           1| 184.0| 340.0| 207.0|2837.0|0.8639947175979614| 0.2431444227695465|         0|
|      [0,0]|           2| 196.0| 356.0| 227.0|2856.0|0.8527408242225647|0.22126929461956024|         0|
+-----------+------------+------+------+------+------+------------------+-------------------+----------+
only showing top 3 rows



clusteredDFs: scala.collection.immutable.IndexedSeq[org.apache.spark.sql.DataFrame] = Vector([spatial_key: struct<col: int, row: int>, column_index: int ... 9 more fields], [spatial_key: struct<col: int, row: int>, column_index: int ... 9 more fields], [spatial_key: struct<col: int, row: int>, column_index: int ... 9 more fields])


## Visualizing Results

In order to visualize the predictions of our model, we must first convert the exploded version of the tile back into an actual tile. We must extract the spatial data from our previous rasterframe, because clusteredRFs is not a RasterFrame (RasterFrames must have a tile column, and this dataframe is full of exploded cell values).

In [47]:
val tiledRFs = (0 to 2).map { i =>
    
    val tlm = fullRFs.apply(i).tileLayerMetadata.left.get

    clusteredDFs.apply(i).groupBy($"spatial_key").agg(
        assembleTile(
            $"column_index", $"row_index", $"prediction",
            tlm.tileCols, tlm.tileRows, ByteConstantNoDataCellType
        )
    ).asRF
} 

tiledRFs: scala.collection.immutable.IndexedSeq[astraea.spark.rasterframes.RasterFrame] = Vector([spatial_key: struct<col: int, row: int>, prediction: rf_tile], [spatial_key: struct<col: int, row: int>, prediction: rf_tile], [spatial_key: struct<col: int, row: int>, prediction: rf_tile])


To render our visualization, we convert to a raster first, and then we define colormaps and render the rasters as images.

In [48]:
val resolutions = Seq((229, 225), (232, 246), (235, 231))

resolutions: Seq[(Int, Int)] = List((229,225), (232,246), (235,231))


 Zip the prediction RFs and their respective resolutions to prepare to render them

In [ ]:
val zipPredsRes = tiledRFs.zip(resolutions)

val predRasters = zipPredsRes.map { pdsRes => pdsRes._1.toRaster($"prediction", pdsRes._2._2, pdsRes._2._1) }

Define our colormaps and then render the rasters.

In [85]:
import geotrellis.raster.render._

val cmap = ColorRamp(0x0ab881FF, 0xe9aa0cFF, 0xe9e10cFF)

val cmap_r = ColorRamp(0xe9e10cFF, 0xe9aa0cFF, 0x0ab881FF)

val clusterColors = IndexedColorMap.fromColorMap(cmap.toColorMap((0 until 3).toArray))
// Reverse the aboce color
val clusterColorsReverse = IndexedColorMap.fromColorMap(cmap_r.toColorMap((0 until 3).toArray))

predRasters.apply(0).tile.renderPng(clusterColors).write("../_images/clust1.png")
predRasters.apply(1).tile.renderPng(clusterColors).write("../_images/clust2.png")
predRasters.apply(2).tile.renderPng(clusterColorsReverse).write("../_images/clust3.png")

import geotrellis.raster.render._
cmap: geotrellis.raster.render.ColorRamp = geotrellis.raster.render.ColorRamp@7c9abac5
cmap_r: geotrellis.raster.render.ColorRamp = geotrellis.raster.render.ColorRamp@18a073b6
clusterColors: geotrellis.raster.render.IndexedColorMap = IndexedColorMap(0xab881ff, 0xe9aa0cff, 0xe9e10cff)
clusterColorsReverse: geotrellis.raster.render.IndexedColorMap = IndexedColorMap(0xe9e10cff, 0xe9aa0cff, 0xab881ff)


Here are the images created by the clustering algorithm:

![](pics/scala_forest.png)

As you can see, the green closely represents forest, with the other two shades representing land that is cleared to some degree. Now let's access more precise statistics about this use by calculating tileHistograms for all three scenes.

In [51]:
val histograms = tiledRFs.map { rf => rf.select(tileHistogram($"prediction")).first() }

histograms: scala.collection.immutable.IndexedSeq[astraea.spark.rasterframes.stats.CellHistogram] = Vector(CellHistogram(CellStatistics(51525,-1,0.0,2.0,0.7990490053372149,0.6090898282632029),List(Bin(0.0,21909), Bin(1.0,18061), Bin(2.0,11555))), CellHistogram(CellStatistics(57072,-1,0.0,2.0,0.802092094196804,0.722729713413809),List(Bin(0.0,27389), Bin(1.0,13589), Bin(2.0,16094))), CellHistogram(CellStatistics(54285,-1,0.0,2.0,0.7863313990973565,0.6986949967418299),List(Bin(0.0,26003), Bin(1.0,13878), Bin(2.0,14404))))


In these examples, it appears that a value of zero (Green in the colormap) represents forest for scenes one and three, and 2 represents forest in scene 2. Let's find the percentage of land covered by forest in these examples by querying our histograms.

In [53]:
// Returns the number of cells that we believe the model marked similarly to forest
val forest1 = histograms.apply(0).itemCount(0.0)
val forest2 = histograms.apply(1).itemCount(2.0)
val forest3 = histograms.apply(2).itemCount(0.0)

forest1: Long = 21909
forest2: Long = 16094
forest3: Long = 26003


In the first scene, a little less than half of the scene is composed of the values that resemble rainforest (according to the model).

Number of forest divided by the total

In [54]:
forest1 / histograms.apply(0).totalCount.toFloat

res12: Float = 0.42521107


Total forest cells for all three scenes, divided by total cells

In [55]:
(forest1 + forest2 + forest3) / 
(histograms.apply(0).totalCount + histograms.apply(1).totalCount + histograms.apply(2).totalCount).toFloat

res13: Float = 0.39295933


In total, it appears that less than 40 percent of the area surveyed in these three scenes is forested. While there are improvements to be made, this algorithm could easily be adapted to survey land of any size. Landsat 8 has a temporal resolution of 16 days, and tracking these numbers over time could give an estimate of the rate of change of deforestation in a certain area, or with enough compute power, the entire Amazon Rainforest.